In [1]:
import pandas as pd

from transliterate import translit

from datetime import datetime

In [3]:
df = pd.read_csv('../data/train.csv')

print(df.shape)
df.head()

(44854516, 10)


,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,Забайкальский Край,Чита,android,10.0
1,4,NaN,NaN,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,Оренбургская область,Оренбург,Android,10.0.0
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,Санкт-Петербург,Санкт-Петербург,android,9.0
3,5,NaN,NaN,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,Свердловская область,Екатеринбург,android,9
4,4,NaN,NaN,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,Московская область,Звенигород,android,6.0.1


# Изучение данных

In [20]:
df['os'].value_counts(dropna=False)

android    35245849
ios         5248627
Android     4253330
iOS          106477
NaN             233
Name: os, dtype: int64

In [19]:
df['osv'].value_counts(dropna=False)

10.0              9447724
11.0              4693227
9.0               4542376
10.0.0            4274736
8.1.0             2603022
                   ...   
21(5.0.2)               1
22(10.0)                1
7.2                     1
20.0                    1
5.0.2 / API-21          1
Name: osv, Length: 255, dtype: int64

In [18]:
df['city'].value_counts(dropna=False)
# маленькие населенные пункты округлять до большого, и ставить флаг маленького в новой колонке

Москва             7644426
NaN                4799992
Санкт-Петербург    3041452
Екатеринбург       2692591
Краснодар          2294394
                    ...   
Янино-2                  1
Berëza                   1
Кощаково                 1
Волковыск                1
Жлобин                   1
Name: city, Length: 2499, dtype: int64

In [17]:
df['oblast'].value_counts(dropna=False)
# найти дубликаты с разными названиями

Москва                  7680369
NaN                     3455278
Свердловская область    3079042
Санкт-Петербург         3070245
Краснодарский край      2998397
                         ...   
Витебская Область            36
Gomelskaya Oblast            29
Могилевская область          28
Гродненская Область          24
Минская Область              10
Name: oblast, Length: 91, dtype: int64

In [16]:
df['created'].value_counts(dropna=False)
# что это за ебаная дата?

1970-01-01 03:00:00    233
2021-08-02 16:26:44    203
2021-08-02 14:03:11    200
2021-08-02 15:11:00    200
2021-08-02 16:13:41    196
                      ... 
2021-07-09 05:56:55      1
2021-08-07 00:50:27      1
2021-08-03 23:44:15      1
2021-07-11 01:55:53      1
2021-07-08 00:16:01      1
Name: created, Length: 1720033, dtype: int64

In [15]:
df['bundle'].value_counts(dropna=False)

com.fugo.wow                                       1678124
net.wargaming.wot.blitz                            1101717
com.openmygame.games.android.wordpizza              902581
com.axlebolt.standoff2                              632030
com.yourstoryinteractive.sails.pirate.adventure     491705
                                                    ...   
best2017translatorapps.english.russian                   1
com.flexsolution.faketextmessage                         1
com.ateam.tanksurvival                                   1
com.mobo.filtertel                                       1
com.fgames.url                                           1
Name: bundle, Length: 85798, dtype: int64

In [14]:
df['subgamecategory'].value_counts(dropna=False)

NaN                        16968425
Puzzle                      4691498
Action                      3412940
Casual                      3242897
Simulation                  2757136
Word                        1827741
Arcade                      1372731
Strategy                    1276408
Role Playing                1202308
Card                        1041830
Board                        883606
Racing                       812515
Adventure                    736299
Tools                        666860
None                         647453
Sports                       576169
Dating                       296753
Shopping                     287733
Music                        286998
Productivity                 222971
Entertainment                218854
Books & Reference            211505
Health & Fitness             176483
Video Players & Editors      151221
Casino                       149003
Trivia                       134411
Music & Audio                115483
Education                   

In [13]:
df['gamecategory'].value_counts(dropna=False)

Games                 24555534
NaN                   16961331
Applications           2742231
Lifestyle               267292
Shopping                184875
Entertainment            31465
Social Networking        28931
Education                27008
Book                     23483
Photo & Video            10235
Utilities                 7560
Music                     4571
IAB9-30                   3474
Health & Fitness          1423
Finance                   1322
Productivity              1314
Travel                     778
Sports                     614
Navigation                 332
Reference                  235
IAB9,IAB9-30,games         226
Graphics & Design           73
Business                    67
Medical                     66
Food & Drink                44
News                        23
Weather                      5
Stickers                     4
Name: gamecategory, dtype: int64

In [21]:
df['Segment'].value_counts()

5    17187506
3    14187054
4    11142080
2     1416245
1      921631
Name: Segment, dtype: int64

# План

1. Почистить данные
2. Спросить откуда битые
3. Привести дату к одной 
4. Разбить бандл на составляющие
5. Смерджить сегменты
6. Добавить дополнительные признаки

## 1. Предобработка данных

In [23]:
# Преобразуем значения колонок к одному виду
df['os'].replace({'Android': 'android', 'iOS': 'ios'}, inplace=True)

In [43]:
df['created'] = pd.to_datetime(df['created'])

# вытащим все возможные признаки из даты
def get_datetime_features(df):
    df['wday'] = df['created'].apply(lambda x: x.weekday())
    df['month'] = df['created'].apply(lambda x : x.month)
    df['day'] = df['created'].apply(lambda x: x.day)
    df['minutes'] = df['created'].apply(lambda x: x.minute)
    df['hours'] = df['created'].apply(lambda x: x.hour)
    df['year'] = df['created'].apply(lambda x: x.year)
    df['week'] = df['created'].apply(lambda x: x.week)
    df['week_of_year'] = df['created'].apply(lambda x: x.weekofyear)
    df['day_of_year'] = df['created'].apply(lambda x: x.dayofyear)
    df['quarter'] = df['created'].apply(lambda x: x.quarter)
    df['is_month_start'] = df['created'].apply(lambda x: x.is_month_start)
    df['is_month_end'] = df['created'].apply(lambda x: x.is_month_end)
    df['is_quarter_start'] = df['created'].apply(lambda x: x.is_quarter_start)
    df['is_quarter_end'] = df['created'].apply(lambda x: x.is_quarter_end)
    return df

df = get_datetime_features(df)

print(df.shape)
df.head()

(44854516, 24)


,Segment,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv,...,hours,year,week,week_of_year,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end
0,4,Games,Racing,com.MadOut.BIG,2021-07-05 18:07:40,MSK+6,Забайкальский Край,Чита,android,10,...,18,2021,27,27,186,3,False,False,False,False
1,4,NaN,NaN,com.easybrain.solitaire.klondike.free,2021-07-10 10:38:42,MSK+2,Оренбургская область,Оренбург,android,10,...,10,2021,27,27,191,3,False,False,False,False
2,5,Games,Arcade,com.orbitalknight.ridiculousfreekick,2021-08-04 13:34:29,MSK,Санкт-Петербург,Санкт-Петербург,android,9,...,13,2021,31,31,216,3,False,False,False,False
3,5,NaN,NaN,tcouchgind.scooterextreme.scooter,2021-08-06 07:35:27,MSK+2,Свердловская область,Екатеринбург,android,9,...,7,2021,31,31,218,3,False,False,False,False
4,4,NaN,NaN,com.FidgetTrading3D.game,2021-08-02 20:43:59,MSK,Московская область,Звенигород,android,6,...,20,2021,31,31,214,3,False,False,False,False


In [38]:
# Преобразуем версии ОС к одному виду
df['osv'] = df['osv'].str.split('.').str[0].str.extract('(\d+)').fillna(-1000).astype(int)

In [44]:
df.drop(columns=['created'], inplace=True)

df.to_csv('../data/main.csv', index=False)

In [2]:
test = pd.read_csv('../data/test.csv')

print(test.shape)
test.head()

(11213629, 9)


,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10.0
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10.0
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14.4.0
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9.0.0
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10.0


In [3]:
# Преобразуем значения колонок к одному виду
test['os'].replace({'Android': 'android', 'iOS': 'ios'}, inplace=True)

In [6]:
test['created'] = pd.to_datetime(test['created'])

# вытащим все возможные признаки из даты
def get_datetime_features(df):
    df['wday'] = df['created'].apply(lambda x: x.weekday())
    df['month'] = df['created'].apply(lambda x : x.month)
    df['day'] = df['created'].apply(lambda x: x.day)
    df['minutes'] = df['created'].apply(lambda x: x.minute)
    df['hours'] = df['created'].apply(lambda x: x.hour)
    df['is_month_start'] = df['created'].apply(lambda x: x.is_month_start)
    df['is_month_end'] = df['created'].apply(lambda x: x.is_month_end)
    return df

test = get_datetime_features(test)

print(test.shape)
test.head()

(11213629, 16)


,gamecategory,subgamecategory,bundle,created,shift,oblast,city,os,osv,wday,month,day,minutes,hours,is_month_start,is_month_end
0,Applications,Shopping,com.allgoritm.youla,2021-09-19 17:31:33,MSK+2,Свердловская область,Екатеринбург,android,10,6,9,19,31,17,False,False
1,NaN,NaN,com.ChocochocoStd.RapBattle,2021-08-03 17:13:17,MSK+2,Ямало-Ненецкий АО,Новый Уренгой,android,10,1,8,3,13,17,False,False
2,NaN,NaN,1387897651,2021-09-17 15:54:00,MSK,Москва,Москва,ios,14,4,9,17,54,15,False,False
3,NaN,NaN,com.cooking.family.diary.fever.food.city.craze...,2021-07-05 23:34:59,MSK,Краснодарский край,Краснодар,android,9,0,7,5,34,23,False,False
4,Applications,Health & Fitness,com.pedometer.stepcounter.tracker,2021-07-08 15:15:21,MSK,Татарстан,Набережные Челны,android,10,3,7,8,15,15,False,False


In [5]:
# Преобразуем версии ОС к одному виду
test['osv'] = test['osv'].str.split('.').str[0].str.extract('(\d+)').fillna(-1000).astype(int)

In [7]:
test.to_csv('../data/test_preproc.csv', index=False)